In [1]:
# Import findspark and initialize. 
import findspark


findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/26 00:33:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")



23/07/26 00:33:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# 2. Create a temporary view of the DataFrame.

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

df
query = """
SELECT year(date) AS year_sold, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Run the SQL query and store the result in a new DataFrame
result_df = spark.sql(query)

# Show the result
result_df.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

result_df = spark.sql(query)

result_df.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
result_df = spark.sql(query)

# Show the result
result_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT ROUND(AVG(view), 2) AS avg_view_rating
FROM home_sales
WHERE price >= 350000
"""

result_df = spark.sql(query)
result_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.26546812057495117 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.

spark = SparkSession.builder.appName("HomeSales").getOrCreate()
spark.catalog.cacheTable("home_sales")


23/07/26 00:39:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query_cached = """
SELECT ROUND(AVG(view), 2) AS avg_view_rating
FROM home_sales
WHERE price >= 350000
"""
result_df_cached = spark.sql(query_cached)
result_df_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


spark.catalog.uncacheTable("home_sales")
start_time_uncached = time.time()
query_uncached = """
SELECT ROUND(AVG(view), 2) AS avg_view_rating
FROM home_sales
WHERE price >= 350000
"""
result_df_uncached = spark.sql(query_uncached)
result_df_uncached.show()


print("--- Uncached Runtime: %s seconds ---" % (time.time() - start_time_uncached))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.6342072486877441 seconds ---
+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Uncached Runtime: 0.17914390563964844 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


df = spark.table("home_sales")
output_path = "/Users/jorgedennis/Downloads/Starter_Code-19/parquet_data_folder"

df.write.partitionBy("date_built").parquet(output_path)


In [19]:
# 11. Read the formatted parquet data.
parquet_data_path = "/Users/jorgedennis/Downloads/Starter_Code-19/parquet_data_folder"
df = spark.read.parquet(parquet_data_path)
df.createOrReplaceTempView("home_sales_partitioned")


In [20]:
# 12. Create a temporary table for the parquet data.


df.createOrReplaceTempView("home_sales_partitioned")


In [21]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

query_parquet = """
SELECT ROUND(AVG(view), 2) AS avg_view_rating
FROM home_sales_partitioned
WHERE price >= 350000
"""

result_df_parquet = spark.sql(query_parquet)
result_df_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

start_time_cached = time.time()
query_cached = """
SELECT ROUND(AVG(view), 2) AS avg_view_rating
FROM home_sales_partitioned
WHERE price >= 350000
"""
result_df_cached = spark.sql(query_cached)
result_df_cached.show()
print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))





+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.3220860958099365 seconds ---
+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Cached Runtime: 0.16011810302734375 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [24]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("true")
else:
    print("false")

false
